<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 10</h1>

<h2>Bad controls in the January 2018 Current Population Survey (CPS)</h2>

<h2>Learning objectives:</h2>

1. More experience with OLS
2. Handy `factor()` function
3. Bad controls may cut off the avenues through which the treatment affects the outcome
4. Balancing omitted variable bias vs. bad controls is difficult

<h2>Bad controls</h2>

Let's examine how controlling for occupation and industry, while feasible to do, changes the estimated coefficients on education and female gender identity. It's probably not a good idea, unless there were a compelling reason to look at within-occupation differences in earnings by education. 

To explore this topic, we'll look at an extract from the January 2018 Current Population Survey (CPS), the annual job tenure supplement. I downloaded the extract from [IPUMS](http://cps.ipums.org), and in the future we'll walk through the IPUMS interface and contents.

In [ ]:
library(tidyverse)
library(haven)

Setting this option removes scientific notation, which can sometimes be a real bummer to read.

In [ ]:
options(scipen = 999)

This extract contains people aged 25-64, which are working ages when education is likely to be completed.

In [ ]:
cpsj18_2564 = read_dta("data/cpsj18_2564_1.dta")
head(cpsj18_2564)

Here is a baseline regression, where we are controlling for 0/1 female gender identity and years of education, and for a set of controls including race/ethnicity, years of experience, years of experience squared, and also years of job tenure:

$$
\ln Y_i = \alpha + \beta^f \ female_i + \beta^e \ educ_i + B \ controls_i + \epsilon_i
$$

We'll run this regression and examine what we find for $\beta^f$ and $\beta^e$:

In [ ]:
cps_reg1 <- lm(logearnweek ~ female 
               + edyrs 
               + factor(raceth) 
               + exper + expersq + jtyears, 
               data = cpsj18_2564)
summary(cps_reg1)

We see a pretty large negative penalty for female workers: $-$37%, and we see a relatively large benefit for an additional year of education: 10.7%. Racial/ethnic minorities see earnings penalties relative to white non-Hispanic people (the baseline group). Log earnings are a parabola opening down in years of experience, which is consistent with typical theory. Years of job tenure is a benefit. 

Now let's explore controlling for occupation `occ`, and for kicks, also for industry `ind` and state of residence `statefip`. The last one is probably not a bad control, come to think of it, unless one believes that education's effect on earning is partially through enhanching geographic mobility, which isn't an impossibility. So let's try just that one first:

In [ ]:
cps_reg2 <- lm(logearnweek ~ female 
               + edyrs 
               + factor(raceth) 
               + exper + expersq + jtyears
               + factor(statefip)
               , data = cpsj18_2564)
summary(cps_reg2)

What do we see? 

In the coefficients on the top variables (i.e., not the state indicators), we do not see many big differences here compared to the first regression. The coefficient on female, $\beta^f$, shows a penalty of about $-$36%; and the coefficient on years of education, $\beta^e$, shows a benefit of additional year is 10.2%.

Let's throw the kitchen sink at things.

In these data, there are almost 500 occupation categories, and there are roughly 250 industries. For details, see the documentation at IPUMS: [OCC documentation](https://cps.ipums.org/cps/codes/occ_20112019_codes.shtml), [IND documentation](https://cps.ipums.org/cps/codes/ind_2014_codes.shtml).

In [ ]:
cps_reg3 <- lm(logearnweek ~ female 
               + edyrs 
               + factor(raceth) 
               + exper + expersq + jtyears
               + factor(occ) 
               + factor(ind) 
               + factor(statefip)
               , data = cpsj18_2564)
summary(cps_reg3)

What do we see here compared to earlier results?

The coefficient on female identity is now about $\beta^f = -0.24$, smaller in magnitude by a factor of about a third. In other words, within occupation-industry-state groupings in the data, female workers earn on average 24% less than observationally identical males.

Meanwhile, the coefficient on years of education has fallen by roughly half, down to $\beta^e = 0.051$. Here too, the story is that within occupation-industry-state groupings of workers, the returns to an additional year of education are smaller than they are overall.

<h2>Discussion</h2>

As is often the case, including more variables $Z$ on the right-hand side of the regression equation often reduces the magnitudes and statistical significance of coefficients we care about, like on a key treatment variable $X$.

<b>The problem</b> is deciding whether the $Z$'s belong there or not. Are they potentially omitted variables that would obscure the true story about $\beta$, the effect of $X$ on $Y$, if they were omitted? Or are they bad controls, which block off a critical channel of causality running between $X$ and $Y$?

An <b>omitted variable</b> is a $Z$ that should be there, and by its omission is biasing the story about the causal effect of $X$ on the outcome $Y$. A classic example would be a parental characteristic that varies with the treatment variable; the thought experiment is: suppose a policy changed $X$ for kids, despite important parental characteristics $Z$ being what they are. What is the likely effect of the policy on $Y$?

A <b>bad control</b> is a $Z$ that shouldn't be there, because controlling for it blocks off a critical pathway through which $X$ causes $Y$. A great example might be $Z = occupation$ and/or industry in a regression of $Y = earnings$ on $X = education$. Education raises earnings in part because it moves people out of engaging in manual labor and into services and other industries. It might be true that a highly educated janitor could be more productive and thus earn more than a poorly educated janitor, but it is the transition from janitor to something like an office worker, doctor, or lawyer that really raises earnings a lot. If we control for occupation and industry in an earnings regression, we will be closing off that pathway of switching occupations or industries, through which we imagine education has a large causal effect on earnings.

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>